In [60]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [61]:
#train.describe()
train = train_.sample(20000)
validate = validate_.sample(10000)
train_.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [62]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')

train = train[train.SUB2.isin([2,3,4,5,7,10])]
validate = validate[validate.SUB2.isin([2,3,4,5,7,10])]
print train.shape
print train['SUB2'].value_counts()

(9512, 62)
4     3319
2     2813
3     1617
7      773
10     728
5      262
Name: SUB2, dtype: int64


In [63]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 0.66t for feature PREG
F-score: 1.07t for feature VET
F-score: 1.22t for feature DAYWAIT
F-score: 3.63t for feature PRIMPAY
F-score: 4.50t for feature ARRESTS
F-score: 5.25t for feature CBSA10
F-score: 5.68t for feature ETHNIC
F-score: 7.22t for feature DETNLF
F-score: 8.22t for feature RACE
F-score: 8.95t for feature SUB3
F-score: 9.70t for feature PRIMINC
F-score: 10.25t for feature LIVARAG
F-score: 12.24t for feature ROUTE3
F-score: 12.79t for feature REGION
F-score: 15.01t for feature GENDER
F-score: 15.42t for feature STFIPS
F-score: 15.91t for feature PSYPROB
F-score: 16.96t for feature FRSTUSE1
F-score: 19.14t for feature MARSTAT
F-score: 22.46t for feature EMPLOY
F-score: 23.94t for feature FRSTUSE3
F-score: 24.24t for feature SERVSETA
F-score: 24.74t for feature HLTHINS
F-score: 27.06t for feature YEAR
F-score: 27.55t for feature CASEID
F-score: 29.08t for feature NOPRIOR
F-score: 29.14t for feature DIVISION
F-score: 29.28t for feature EDUC
F-score: 33.35t for feature

In [64]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'PREG','VET','DAYWAIT','PRIMPAY','ARRESTS','CBSA10','ETHNIC','DETNLF','RACE','SUB3','PRIMINC','LIVARAG','ROUTE3','REGION','GENDER','STFIPS','PSYPROB','FRSTUSE1','MARSTAT','EMPLOY','FRSTUSE3','SERVSETA','HLTHINS','YEAR','CASEID','NOPRIOR','DIVISION','EDUC','DETCRIM','FREQ2','METHUSE','NUMSUBS','FREQ1','PSOURCE','FREQ3','ROUTE1','SUB1','AGE','DSMCRIT','IDU','ROUTE2','ALCDRUG','FRSTUSE2',

In [65]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU','SUB3','ROUTE3',
               'FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((9512, 23), (4794, 23))

In [66]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'YEAR', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'REGION', 'NOPRIOR', 'DIVISION', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ2', 'FRSTUSE2']


In [67]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((9512, 184), (4794, 184))

In [68]:
# Logistic Regression
logreg = LogisticRegression(C=1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


69.63


In [69]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


62.83


In [70]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


65.77


In [71]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100, max_depth=20)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

67.88


In [72]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


63.93


In [73]:
# Linear SVC
linear_svc = LinearSVC(C=1)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


69.27


In [74]:
import xgboost as xgb
xgb_boost = xgb.XGBClassifier(seed=1850, n_jobs=-1)
xgb_boost.fit(X_train, Y_train) 
acc_xgb = round(xgb_boost.score(X_validate, Y_validate) * 100, 2)
print acc_xgb

68.71


In [75]:
print 'predict-top6-sub2-allcols-sample10000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'XGBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_xgb]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-top6-sub2-allcols-sample10000
                        Model  Validate Set Score
0         Logistic Regression               69.63
4                  Linear SVC               69.27
5                     XGBoost               68.71
1               Random Forest               67.88
3  Stochastic Gradient Decent               63.93
2                  Perceptron               62.83
